# Amazon EagleEye Data Analysis

This notebook provides an introduction to interactive analysis of the data captured by SageMaker EagleEye. It is organized in order of training phases: initialization, training, and finalization.

## Table of Contents

* [1. Install dependencies](#1)<br>
* [2. Preparing data for analysis](#2)<br>
* [3. Analysis by training job phase](#analysis)<br>
    * [3.1 Initialization](#initialization)<br>
    * [3.2. Training loop](#training)<br>
        * [3.2.1 Data loading](#dataloading)<br>
    * [3.2. Finalization](#finalization)<br>

<a class="anchor" id="2"></a>
## 2. Preparing data for analysis

Import Python libraries for reading data and visualization.

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

###  Access Profiler Data: System Metrics and Algorithm (Framework) Metrics

Once the training job initiates, SageMaker Profiler starts collecting system and framework metrics. The `smdebug` library provides profiler analysis tools that enable you to access and analyze the profiling data. The following code cells are to set up a `TrainingJob` object to retrieve the system and framework metrics when they become available in the default S3 bucket. Once the metrics are available, you can query, plot, and analyze the profiling metrics data throughout this notebook. Specify the training job name in the following cell. The job name is provided at the end of the training example notebooks.

#### Specify the training job name and the region name

In [ ]:
training_job_name = "smprofiler-report-snmg-mirror-2020-09-23-05-37-57-786"
region = 'us-east-1'

In [ ]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

smdebug provides a class PandasFrame that converts profiler data into Pandas frames.

In [ ]:
from smdebug.profiler.analysis.utils.profiler_data_to_pandas import PandasFrame

pf = PandasFrame(tj.profiler_s3_output_path)

Following functions retrieve all system and framework metrics.

In [ ]:
system_metrics_df = pf.get_all_system_metrics()
framework_metrics_df = pf.get_all_framework_metrics(selected_framework_metrics=['Step:ModeKeys.TRAIN', 'Step:ModeKeys.GLOBAL'])

<a id='analysis'></a>
## 3. Analysis by training job phase

#### Initial setup.

Here we will set up some helper functions and python profile analysis to be used in the analysis later in the section.

In [ ]:
import os
from IPython.display import IFrame, display, Markdown
from smdebug.profiler.python_profile_utils import StepPhase, PythonProfileModes
from smdebug.profiler.analysis.utils.python_profile_analysis_utils import Metrics
from smdebug.profiler.analysis.python_profile_analysis import PyinstrumentAnalysis, cProfileAnalysis
from smdebug.profiler.analysis.utils.pandas_data_analysis import PandasFrameAnalysis, StatsBy, Resource

In [ ]:
from IPython.display import display, HTML, Markdown, Code, Image
def pretty_print(df):
    raw_html = df.to_html().replace("\\n","<br>").replace('<tr>','<tr style="text-align: left;">')
    return display(HTML(raw_html))

In [ ]:
pf_analysis = PandasFrameAnalysis(system_metrics_df, framework_metrics_df)

In [ ]:
python_stats_dir = "python_stats"
os.makedirs(python_stats_dir, exist_ok=True)
general_metrics_config = eval(tj.profiler_config["ProfilingParameters"].get("GeneralMetricsConfig", "{}"))
use_pyinstrument = False

In [ ]:
def display_python_profile_stats(stats):
    if use_pyinstrument:
        if stats:
            for step_stats in stats:
                display(IFrame(src=step_stats.html_file_path, width= 1000, height=500))
    else:
        if stats:
            stats.print_top_n_functions(Metrics.CUMULATIVE_TIME, n=10)

In [ ]:
python_analysis_class = PyinstrumentAnalysis if use_pyinstrument else cProfileAnalysis
python_analysis = python_analysis_class(local_profile_dir=python_stats_dir, s3_path=tj.profiler_s3_output_path)
step_stats_df = python_analysis.list_profile_stats()
step_stats_df.head()

#### Identifying the framework

Some of the analysis can be framework-specific. Since the profiler data or the training job detail above does not indicate which framework was used, below we try to identify the framework used for training.

In [ ]:
framework = "tensorflow"

process_list = framework_metrics_df["process"].unique()
if 'Step:ModeKeys.GLOBAL' in process_list:
    framework = "pytorch"

#### Identifying the time interval of each phase

The function `get_job_statistics()` shows training start and end time, duration of the initialization, training, and finalization stages.

In [ ]:
job_statistics = pf_analysis.get_job_statistics()

initialization_start = job_statistics['start_time']
initialization_end = job_statistics['training_loop_start']

training_start = job_statistics['training_loop_start']
training_end = job_statistics['training_loop_end']

finalization_start = job_statistics['training_loop_end']
finalization_end = job_statistics['end_time']

<a id='initialization'></a>
## 3.1 Initialization

This phase of the training job marks the time interval from the start of the training job in SageMaker to the start of the training loop.

**Python profiling**

Let's look at the python profiling stats for functions that were called during initialization.

In [ ]:
stats = python_analysis.fetch_pre_step_zero_profile_stats()
display_python_profile_stats(stats)

<a id='training'></a>
## 3.2 Training loop

This phase of the training job indicates the time interval between the start and the end of the training loop. It includes the data loading process, the forward and backward pass, and synchronization.

#### Utilization histograms

MetricHistogram computes a histogram on GPU and CPU utilization values. Bins are between 0 and 100. Good system utilization means that the center of the distribtuon should be between 80 to 90. 

The following cell will plot the histograms per metric. In order to only plot specific metrics define the list  `select_dimensions` and `select_events`. A dimension can be CPUUtilization, GPUUtilization, GPUMemoryUtilization IOPS. If no event is specified then for CPU uiltization histogram for each single core and total cpu usage will be plotted. In case of GPU, it will visualize utilization and memory for each GPU. In case of IOPS it will plot io-wait time per cpu. If `select_events` is specified then only metrics that match the name in `select_metrics` will be shown. If neither `select_dimensions` nor `select_events` all available metrics will be visualized. One can also specify a start and endtime.

In [ ]:
from smdebug.profiler.analysis.notebook_utils.metrics_histogram import MetricsHistogram

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

metrics_histogram = MetricsHistogram(system_metrics_reader)
metrics_histogram.plot(starttime=0, 
                       endtime=system_metrics_reader.get_timestamp_of_latest_available_file(), 
                       select_dimensions=["CPU", "GPU"],
                       select_events=["total"])

#### Step durations over time

SageMaker Profiler records the durations of each step, which is the time spent in one forward and backward pass. The following code cell plots step durations (y-axis) over training job duration (x-axis). Typically we would expect the step duration to be very similar across the training run. Signficant outliers are an indication of a bottleneck. `StepTimelineChart` helps to identify if such outliers happen in regular intervals. Following image shows an example, where the step duration mostly lasts about 200 to 250ms but every 10th step a spike occurs where step duration is significantly higher (600-800ms).

In [ ]:
tj.wait_for_framework_profiling_data_to_be_available()

from smdebug.profiler.analysis.notebook_utils.step_timeline_chart import StepTimelineChart

framework_metrics_reader = tj.get_framework_metrics_reader()
framework_metrics_reader.refresh_event_file_list()

view_step_timeline_chart = StepTimelineChart(framework_metrics_reader)

#### Python profiling

If detailed profiling was enabled, then python profiling was done on the steps specified for detailed profiling. Let's look at the python profiling stats for functions that were called during the first step profiled. We will look at the stats for each phase of the step. 

If you profiled with cProfile, you will see the function stats for the top 10 functions with greatest cumulative time spent. If you prefer to use a different metric or a different `n`, feel free to modify the `display_python_profile_stats` function defined at the beginning of the section.

If you profiled with Pyinstrument, you will see the stats separated by each phase of the step.

In [ ]:
if general_metrics_config == {}:
    step = 9
else:
    step = int(general_metrics_config["StartStep"])
phase = StepPhase.STEP_START
mode = PythonProfileModes.GLOBAL if framework == "pytorch" else PythonProfileModes.TRAIN
python_step_stats = python_analysis.fetch_profile_stats_by_step(
    start_step=step, 
    end_step=step + 1, 
    start_phase=phase, 
    end_phase=phase,
    mode=mode
)
display_python_profile_stats(python_step_stats)

If detailed profiling was enabled and cProfile was used as the Python profiler, we can also just look at the stats for the training loop as a whole.

In [ ]:
if use_pyinstrument:
    print("Pyinstrument was used as the Python profiler, no training loop stats are available.")
else:
    python_job_stats = python_analysis.fetch_profile_stats_by_job_phase()
    training_loop_stats = python_job_stats["training_loop"]
    display_python_profile_stats(training_loop_stats)

#### CPU/GPU utilization

SageMaker Profiler also records system metrics (resource utilization) for the training job. The following timeline charts depict the utilization per core and GPU. It shows the last 1000 datapoints. You can inspect previous datapoints by zooming out of the chart.

In [ ]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
framework_metrics_reader.refresh_event_file_list()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts  = TimelineCharts(system_metrics_reader, 
                                       framework_metrics_reader,
                                       select_dimensions=["CPU", "GPU"],
                                       select_events=["total"])

You can use the BoxSelectTool to make a selection in the timeline chart.

<img src='images/boxselect.png' width="840" height="180" border="10" />


The following code cell identifies which time annotations have been recorded in the training job for the selected timerange:

In [ ]:
# Note change index range below with selected index range from above cell
view_timeline_charts.find_time_annotations([300,310]) 

Following cell creates a detailed view of framework metrics for the selected timerange. To avoid issues with out of memory, it will only plot the first 1000 datapoints.

In [ ]:
# Note change index range below with selected index range from above cell
view_timeline_charts.plot_detailed_profiler_data([300,310])

#### Identifying CPU bottlenecks

A heatmap is a representation where each row corresponds to one metric (CPU core and GPU utilizations) and x-axis is the duration of the training job. It allows to more easily spot CPU bottlenecks e.g. if utilization on GPU is low but a utilization of one or more cores is high. 

In the below heatmap, Yellow indicates maximum utilization, purple means that utilization was 0. GPUs have frequent stalled cycles where utilization is dropping to 0 while at the same time utilization on CPU cores is at a maximum. This is a clear indication of a CPU bottleneck where GPUs are waiting for the data to arrive. Such a bottleneck can be caused for instance by a compute-heavy pre-processing.

In [ ]:
from smdebug.profiler.analysis.notebook_utils.heatmap import Heatmap

system_metrics_reader.refresh_event_file_list()
view_heatmap = Heatmap(system_metrics_reader, plot_height=450)

#### Merging all the trace events 

The profiler captures events from different sources, such as framework, debugger hook, Horovod (if applicable). Putting them together, aligning the events and visualizing, will give an idea of events occurring in different components of the training job, and identify potential issues. The merge_timeline() API below combines the different types of trace event files into one trace event file, which can then be viewed in the browser.

Below, we merge the events that occurred between step 1 and 10. You can modifying this step interval, or alternatively, merge events within a time interval.

In [ ]:
from smdebug.profiler.analysis.utils.merge_timelines import MergedTimeline, MergeUnit

start_step, end_step = 1, 10
combined_timeline = MergedTimeline(tj.profiler_s3_output_path, output_directory="./")
combined_timeline.merge_timeline(start_step, end_step, unit=MergeUnit.STEP)

You can download the above file and load it in a browser (chrome://tracing).

Next, since data loading is one of the most compute-intensive parts of the training process, below, we dive deep into analysing profiler data related to data loading.

<a id='dataloading'></a>
## 3.2.1 Data loading

In [ ]:
if framework == "pytorch":
    display(Markdown("""### Data loading analysis for PyTorch\nWe will be instantiating the PT_dataloader_analysis class and invoking the methods in this class to perform the analysis.
    The PT_dataloader_analysis class looks for dataloader events that are specific to PyTorch. Running this analysis for other frameworks will yield incorrect data.
    """))
    
    from smdebug.profiler.analysis.utils.pytorch_dataloader_analysis import PT_dataloader_analysis
    pt_analysis = PT_dataloader_analysis(pf)
    
    display(Markdown("""### Analysis of DataloaderIter initializations.\nThe following cell analyzes\n1. Which type of Dataloader iterators were initialized.\n2. The number of workers per iterator.\n3. Log the status of pin_memory.\n4. Number of times the iteratos were initilized during training.  In PyTorch, 
    iterators are initialized every time iterations over the dataset is to be started. 
    (i.e. typically at the beginning of every epoch.) During initialization, PyTorch 
    spins of worker processes depending upon the configured number of workers, 
    establishes data queue to fetch data and pin_memory thread, if pin_memory is set.\nThe analysis outputs the median and maximum duration for these initializations. If 
    there are outliers, (i.e duration is greater than 2 * median), the function prints 
    the start and end times for those durations. These can be used to inspect system 
    metrics during those time intervals.
    """))
    
    pt_analysis.analyze_dataloaderIter_initialization()
    
    display(Markdown("""### Analysis of Dataloader worker processes\nIn PyTorch, every time DataLoaderIterator is initalized, it spins of the worker processes that feed the data to iterator through attached data queue. These worker processes have lifetime similar to that of DataLoaderIterator.
    The following analysis shows\n1. The number of worker processes that were spun off during the entire training.\n2. Median and maximum duration for the worker processes.\n3. Start and end time for the worker processes that are outliers.
    """))
    
    md = pt_analysis.analyze_dataloaderWorkers()
    
    display(Markdown("""### Analysis of DataLoaderIter::GetNext\nIn PyTorch, the GetNext method is responsible for fetching the data from worker 
    processes through the data queue.\nThese calls are run in the main training thread.\nThe analysis of these events show\n1. Number of GetNext calls made during the training.\n2. Median and maximum duration in micoseconds for GetNext calls.\n3. Start time, End time, duration and worker id for the outlier GetNext call duration.    
    """))
    
    md = pt_analysis.analyze_dataloader_getnext()
    if md is not None:
        pretty_print(md)
    
    display(Markdown("""### Analyze a specific outlier in DataLoaderIter::GetNext\nTo analyze specific outlier, select the row location corresponding to the outlier 
    that we want to analyze and run the following cells. Currently we will be plotting 
    the first outlier.
    """))

In [ ]:
if framework == "pytorch":
    display(Markdown("""### Analysis of training activity for each batch of data.\nSince, we have the start and end times of all the GetNext calls, we can find the amount of time spent by the training script on one batch of data.\n1. We will get the time spent on each data batch by finding the difference between start time of current GetNext call and subsequent GetNext call. Let's call it 'BatchTime_in_seconds'\n2. We will find the outliers in 'BatchTime_in_seconds' and start and end time for those outliers.\n3. Obtain the framework and system metrics during those timestamps. This will indicate where the time was spent.
    \n\n### Analyze the BatchTime_in_seconds\nFollowing cell\n1. plots the BatchTime_in_seconds\n2. Prints the median 'BatchTime_in_seconds'\n3. Creates a dataframe that contains outliers.
    """))
    
    md_batch = pt_analysis.analyze_batchtime()
    if md_batch is not None:
        pretty_print(md_batch)
    
    display(Markdown("""### Analyze a specific outlier in BatchTime_in_seconds\nTo analyze specific outlier, select the row location corresponding to the outlier 
    that we want to analyze and run the following cells. Currently we will be plotting 
    the first outlier.
    """))
    
    # Plotting the TimeLine charts for the first outlier reported in the above dataframe.
    if md_batch is not None and md_batch.size > 0:
        index = 0
        start_timestamp = pf.convert_datetime_to_timestamp(md_batch.loc[index]['previous_batch_start'])
        end_timestamp = pf.convert_datetime_to_timestamp(md_batch.loc[index]['start_time'])
        view_timeline_charts=pt_analysis.plot_the_window(start_timestamp, end_timestamp)

#### 3.2.1.1 Tensorflow Analysis

In [ ]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()
system_metrics_reader = tj.get_systems_metrics_reader()

tj.wait_for_framework_profiling_data_to_be_available()
framework_metrics_reader = tj.get_framework_metrics_reader()
framework_metrics_reader.refresh_event_file_list()

from smdebug.profiler.analysis.utils.profiler_data_to_pandas import PandasFrame

pf = PandasFrame(tj.profiler_s3_output_path)
system_metrics_df = pf.get_all_system_metrics()

step_metrics_df = pf.get_all_framework_metrics(selected_framework_metrics=['Step:ModeKeys.TRAIN'])

step_metrics_df.head(10)

In [ ]:
## We filter events of type GeneratorDatasetOp::Dataset::Iterator::GetNext.\n
## This is the event that performs data loading for a given train step.\n
## Measuring the time span of these events are useful indicators of how efficient the data loading is.

iterator_metrics = pf.get_all_framework_metrics(selected_framework_metrics=['tensorflow::data::GeneratorDatasetOp::Dataset::Iterator::GetNext'])
iterator_metrics.head(10)

In [ ]:
## Prints the time taken by first 10 time steps to perform data loading.
## It also lists the number of workers doing the data loading.

for i in range(10):
    cur_step = step_metrics_df.iloc[i]
    step_pid = cur_step["pid"]
    step_tid = cur_step["tid"]
    cur_step_number = cur_step["step"]
    step_start_time = cur_step["start_time_us"]
    step_end_time = cur_step["end_time_us"]
    step_duration = step_end_time - step_start_time

    data_iter_cur_step = iterator_metrics.loc[(iterator_metrics['start_time_us'] >= step_start_time) & (iterator_metrics['end_time_us'] <= step_end_time)]
    num_workers = data_iter_cur_step.shape[0]
    dl_time_mean = data_iter_cur_step["end_time_us"].mean() - data_iter_cur_step["start_time_us"].mean()
    print(f"Step number: {cur_step_number}, time taken by train step: {step_duration}, Number of dataloading workers: {num_workers}, Average dataloading time of all the workers: {dl_time_mean} microseconds")


<a id='finalization'></a>
## 3.3 Finalization

This phase of the training job marks the time interval from the end of the training loop to the end of the training job in SageMaker.

**Python profiling**

Let's look at the python profiling stats for functions that were called during finalization.

In [ ]:
stats = python_analysis.fetch_post_hook_close_profile_stats()
display_python_profile_stats(stats)